In [54]:
import mezo
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import scipy.sparse as ss

### Good Resources
- [Notes by László Oroszlány](http://oroszl.web.elte.hu/mezo/OL_lecture_notes-1.pdf)
- [Non Equilibrium Green’s Functions for Dummies](https://arxiv.org/pdf/cond-mat/0210519.pdf)
- [Green’s Functions in Quantum Physics, E.N. Economou (Springer, 2005)]()
- [Theory of Quantum Transport at Nanoscale, Dmitry A. Ryndyk Chapter 3. (Springer, 2016)]()

## Task: BHZ model dirty edge local density
The Hamiltonian is
$$
\hat H_{BHZ} = \begin{bmatrix}
\hat H_{QWZ} & 0 \\
0 & \hat H^{\dagger}_{QWZ}
\end{bmatrix}.
$$
In QWZ, we have
$$
\hat T_{x/y} = \frac{\hat\sigma_z + \hat\sigma_{x/y}}{2}, \hat U = \Delta\hat\sigma_z
$$
My tasks:
- Calculate the site resolved local density of states for a BHZ wire with a scatterer located at the edge of the sample.
- Take simple potetial scattering i.e. $V\propto I_4$
- Take completely random scatterer.

The full Hamiltonian of the QWZ system is
$$
\begin{align*}
\hat H_{QWZ} = \begin{bmatrix}
\ddots &                 &                &                  &                 &        &      \\
\dots  &  H_0^L          & H_1^L          &                  &                 &        &\dots \\
\dots  &  H_1^{L\dagger} & H_0^L          &         H_1^L    &                 &        &\dots \\
\dots  &                 & H_1^{L\dagger} &         H_0^L    &  H_1^{LS}       &        &\dots \\
\dots  &                 &                & H_1^{LS\dagger}  &  H_0^S          & H_1^S  &\dots \\
\dots  &                 &                &                  &  H_1^{S\dagger} & H_0^S  &\dots \\
       &                 &                &                  &                 &        &\ddots
\end{bmatrix}
\end{align*}
$$

$$
\hat H_0^L =
\begin{bmatrix}
\ddots &                    &                     &                    &               & \\
       & U\hat\sigma_z      &  \hat T_y           &                    &               & \\
       & \hat T^{\dagger}_y &  U\hat\sigma_z      & \hat T_y           &               & \\
       &                    &  \hat T^{\dagger}_y & U\hat\sigma_z      & \hat T_y      & \\
       &                    &                     & \hat T^{\dagger}_y & U\hat\sigma_z & \\
       &                    &                     &                    &               & \ddots
\end{bmatrix}
$$

$$
\hat H_1^L = \hat H_1^S = 
\begin{bmatrix}
\ddots &          &          &          &          & \\
       & \hat T_x &          &          &          & \\
       &          & \hat T_x &          &          & \\
       &          &          & \hat T_x &          & \\
       &          &          &          & \hat T_x & \\
       &          &          &          &          & \ddots
\end{bmatrix}
$$

$$
H^0_S =
\begin{bmatrix}
\ddots &                    &                     &                    &               & \\
       & U\hat\sigma_z      &  \hat T_y           &                    &               & \\
       & \hat T^{\dagger}_y &  U\hat\sigma_z      & \hat T_y           &               & \\
       &                    &  \hat T^{\dagger}_y & U\hat\sigma_z      & \hat T_y      & \\
       &                    &                     & \hat T^{\dagger}_y & U\hat\sigma_z & \\
       &                    &                     &                    &               & \ddots
\end{bmatrix}
+ \hat R,
$$
where $\hat R\propto I_{2N}$ is a random matrix and $N$ is the number of rows in the lattice

In [56]:
s0 = np.array([[1., 0.], [0., 1.]])
sx = np.array([[0., 1.], [1., 0.]]) 
sy = np.array([[0., -1.0j],[1.0j, 0.]])
sz = np.array([[1., 0.], [0., -1.]])

In [75]:
Ty = 0.5*(sz+sy)
Tx = 0.5*(sz+sx)
U = -1

In [108]:
# Size of the scattering region
nrows = 5
ncols = 5

In [113]:
def get_H0L(nrows):
    """
    Calculate H_0^L 
    """
    H0 = ss.kron(np.eye(nrows), U*sz) + ss.kron(np.diag(np.ones(nrows-1), 1), Ty) + ss.kron(np.diag(np.ones(nrows-1), -1), Ty.T.conj())
    return H0

def get_H1L_H1S(nrows):
    """
    Calculate H_1^L and H_1^S
    """
    return ss.kron(np.eye(nrows), Tx)

def get_H1LS(nrows, c=1):
    """
    Calculate H_1^{LS}
    c -> coupling constant
    """
    return ss.kron(np.eye(nrows), c*Tx)

def get_H0S(nrows, nrowsdirty, V):
    """
    Calculate H_0^S.
    nrowsdirty -> How many rows should have noise
    V -> Noise scale
    """
    H0 = ss.kron(np.eye(nrows), U*sz) + ss.kron(np.diag(np.ones(nrows-1), 1), Ty) + ss.kron(np.diag(np.ones(nrows-1), -1), Ty.T.conj())
    R1 = ss.kron(np.eye(nrowsdirty), V*s0)
    R2 = ss.kron(np.eye(nrows-nrowsdirty), s0)
    H0 = H0 + ss.bmat([[R2, None], [None, R1]])
    return H0

def build_qwz_system(nrows, ncols, nrowsdirty, V, c):
    # Create scattering region
    H0S = get_H0S(nrows, nrowsdirty, V)
    H1S = get_H1L_H1S(nrows)
    H1LS = get_H1LS(nrows, c)
    
    # This is the purely scattering region, we need to attach the lead to this
    HScatter = ss.kron(np.eye(ncols-1), H0S)\
    + ss.kron(np.diag(np.ones(ncols-2), 1), H1S)\
    + ss.kron(np.diag(np.ones(ncols-2), -1), H1S.T.conj())
    
    return HScatter

# TODO: Get the Green function
def get_gf():
    # TODO: Attach Lead
    lead = mezo.lead(H0L, H1L,)

# Sandbox area

In [93]:
H = ss.kron(np.eye(2), U*sz)

In [94]:
H.toarray()

array([[-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [101]:
ss.kron(np.eye(2), Tx).toarray()

array([[ 0.5,  0.5,  0. ,  0. ],
       [ 0.5, -0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0.5],
       [ 0. ,  0. ,  0.5, -0.5]])

In [102]:
ss.kron(np.diag(np.ones(2), 1), Tx).toarray()

array([[ 0. ,  0. ,  0.5,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0.5, -0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0.5, -0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [103]:
get_H0L(10).toarray().shape

(20, 20)

In [104]:
V = 6
nrowsdirty = 2
nrows=5
R1 = ss.kron(np.eye(nrowsdirty), V*s0)
R2 = ss.kron(np.eye(nrows-nrowsdirty), s0)
ss.bmat([[R2, None], [None, R1]]).toarray()

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 6., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 6., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 6., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 6.]])

In [105]:
A = B = 2

In [110]:
def getA():
    A = 3\
    + 3\
    + 2
    return A

In [111]:
getA()

8